
![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)





[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/finance-nlp/04.1.Training_Financial_Binary_Classifier.ipynb)

# Train Domain-specific Binary Classifiers

In this notebook, you will learn how to use Spark NLP and Finance NLP to train custom binary classification models.

Here we will train a sample model to classify if a clause relevant `work_experince` or `other` in a finance document.

# Installation

In [ ]:
! pip install -q johnsnowlabs

## Automatic Installation
Using my.johnsnowlabs.com SSO

In [2]:
from johnsnowlabs import nlp, finance

# nlp.install(force_browser=True)

## Manual downloading
If you are not registered in my.johnsnowlabs.com, you received a license via e-email or you are using Safari, you may need to do a manual update of the license.

- Go to my.johnsnowlabs.com
- Download your license
- Upload it using the following command

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

- Install it

In [ ]:
nlp.install()

# Starting

In [ ]:
spark = nlp.start()

## Introduction

Although John Snow Labs provides mnay pretrained models that cover different applications in the financial domain, there are still problems that are specific to companies or practitioners. For such cases, it is possible to train a new custom model using Finance NLP annotators:

- `ClassifierDLApproach`: Trains a multilabel model (predicts one class out of a predefined set of classes) or binary classification
- `MultiClassifierDLApproach`: Trains a mutilabel model (predicts one or more classes for each document)

## Training Binary Models with `ClassifierDLApproach`

The `ClassifierDLApproach` annotator trains a multiclass model or binar models, where the predictions is one category out of a predifined set of categories that are present in the training data.

The input to are Sentence Embeddings such as the state-of-the-art [UniversalSentenceEncoder](https://nlp.johnsnowlabs.com/docs/en/transformers#universalsentenceencoder), [BertSentenceEmbeddings](https://nlp.johnsnowlabs.com/docs/en/transformers#bertsentenceembeddings) or [SentenceEmbeddings](https://nlp.johnsnowlabs.com/docs/en/annotators#sentenceembeddings).

To train a custom model, you need labeled data with at least the columns

```
| TEXT | LABELS (list) |
```

### Loading the data

In [6]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Finance/data/finance_binary_clf.csv

In [7]:
import pandas as pd
df = pd.read_csv('./finance_binary_clf.csv')
print(f"Shape of the full dataset: {df.shape}")

Shape of the full dataset: (313, 2)


In [8]:
df.head()

,text,label
0,Health LLC a healthcare data management solut...,work_experience
1,judgment including the involvement of tax pro...,work_experience
2,public information Accordingly investors shou...,work_experience
3,Over the next few years PCC acquired other com...,work_experience
4,s team with the Company since December 2018\na...,work_experience


In [9]:
df.tail()

,text,label
308,\nWe use open source software in our offerings...,other
309,\nOn February 18 2020 through our wholly owned...,other
310,sor company VASCO Corp entered the data securi...,other
311,a London based provider of insurance tax comp...,other
312,The Bank J. Safra Sarasin Ltd (previously name...,other


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    313 non-null    object
 1   label   313 non-null    object
dtypes: object(2)
memory usage: 5.0+ KB


In [11]:
df.value_counts("label")

label
other              175
work_experience    138
dtype: int64

> We will use a sample from this dataset to avoid making the training process faster (to illustrate how to perform them). Use the full dataset if you want to experiment with it and achieve more realistic results. 
>
> The sample has size of 314 observations only, please keep in mind that this will impact the accuracy and generalization capabilities of the model. Since the dataset is smaller now, we use 90% of it to train the model and the other 10% for testing.

In [12]:
data = spark.createDataFrame(df)

# If you have a single dataset, then split it or else you can load the test dataset the same way that you load the train data.
train, test = data.randomSplit([0.9, 0.1], seed=42)

In [13]:
train.show(truncate=50)

+--------------------------------------------------+---------------+
|                                              text|          label|
+--------------------------------------------------+---------------+
|
 
Chief Executive Officer and Director
Jeffrey...|work_experience|
|
In 1996 we expanded our computer security busi...|          other|
|
In January 2020 we acquired 100 of the outstan...|          other|
|
In fiscal 2019 we acquired 100 of the equity o...|          other|
|
Lior
Kohavi joined Cyren in June 2013 as Chief...|work_experience|
|
On October 15 2018 we acquired tCell io Inc tC...|          other|
|
The fragmented nature of our market provides a...|          other|
|
have constructed our own Bitcoin mining facili...|          other|
| 000 Mr Lowrey
was also eligible for a cash and...|work_experience|
| 13a 15 f of the Exchange Act Under the supervi...|work_experience|
| Health LLC a healthcare data management soluti...|work_experience|
| Prior to his appointment as Corp

In [14]:
from pyspark.sql.functions import col

test.groupBy("label").count().orderBy(col("count").desc()).show()

+---------------+-----+
|          label|count|
+---------------+-----+
|          other|   16|
|work_experience|    8|
+---------------+-----+



### Train With Universal Encoder

Universal Encoder is a state-of-the-art architecture to create vector representations of text. We already have a pretrained model that can be used instead of training both embeddings and the classifier (but it could also be done). 

The pretrained model was trained and optimized for greater-than-word length text, such as sentences, phrases or short paragraphs. It is trained on a variety of data sources and a variety of tasks with the aim of dynamically accommodating a wide variety of natural language understanding tasks. The input is variable length English text and the output is a 512 dimensional vector.

In [15]:
document_assembler = (
    nlp.DocumentAssembler()
    .setInputCol("text")
    .setOutputCol("document")
    .setCleanupMode("shrink")
)

embeddings = (
    nlp.UniversalSentenceEncoder.pretrained()
    .setInputCols("document")
    .setOutputCol("sentence_embeddings")
)

classifierdl = (
    finance.ClassifierDLApproach()
    .setInputCols(["sentence_embeddings"])
    .setOutputCol("class")
    .setLabelColumn("label")
    .setMaxEpochs(30)
    .setEnableOutputLogs(True)
    .setOutputLogsPath("binary_use")
    .setLr(0.001)
    .setBatchSize(4)
)

clf_pipeline = nlp.Pipeline(stages=[document_assembler, embeddings, classifierdl])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [16]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 225 ms, sys: 38.6 ms, total: 263 ms
Wall time: 34.1 s


In [17]:
import os
log_file_name = os.listdir("binary_use")[0]

with open("binary_use/"+log_file_name, "r") as log_file :
    print(log_file.read())

Training started - epochs: 30 - learning_rate: 0.001 - batch_size: 4 - training_examples: 289 - classes: 2
Epoch 0/30 - 0.88s - loss: 39.652786 - acc: 0.8055556 - batches: 73
Epoch 1/30 - 0.57s - loss: 25.961624 - acc: 0.96875 - batches: 73
Epoch 2/30 - 0.53s - loss: 24.500841 - acc: 0.9826389 - batches: 73
Epoch 3/30 - 0.53s - loss: 23.89011 - acc: 0.9826389 - batches: 73
Epoch 4/30 - 0.51s - loss: 23.673073 - acc: 0.9861111 - batches: 73
Epoch 5/30 - 0.51s - loss: 23.552307 - acc: 0.9861111 - batches: 73
Epoch 6/30 - 0.53s - loss: 23.472414 - acc: 0.9895833 - batches: 73
Epoch 7/30 - 0.52s - loss: 23.41457 - acc: 0.9895833 - batches: 73
Epoch 8/30 - 0.54s - loss: 23.37021 - acc: 0.9895833 - batches: 73
Epoch 9/30 - 0.49s - loss: 23.334728 - acc: 0.9930556 - batches: 73
Epoch 10/30 - 0.49s - loss: 23.305414 - acc: 0.9930556 - batches: 73
Epoch 11/30 - 1.05s - loss: 23.2805 - acc: 0.9930556 - batches: 73
Epoch 12/30 - 0.58s - loss: 23.258955 - acc: 0.9930556 - batches: 73
Epoch 13/30 -

### Test the trained model

In [18]:
preds = clf_pipelineModel.transform(test)

In [19]:
preds_df = preds.select("label", "text", "class.result").toPandas()
preds_df.head()

,label,text,result
0,work_experience,\nPursuant to the requirements of the Securiti...,[work_experience]
1,other,\nWe plan to continue to grow our business by ...,[other]
2,work_experience,Goren is retained\nby the Company through an A...,[work_experience]
3,work_experience,"In December 2005, Ducati returned to Italian o...",[other]
4,work_experience,Weaver was Senior Vice President and Deputy Ge...,[work_experience]


In [20]:
# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [21]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))

                 precision    recall  f1-score   support

          other       0.93      0.88      0.90        16
work_experience       0.78      0.88      0.82         8

       accuracy                           0.88        24
      macro avg       0.86      0.88      0.86        24
   weighted avg       0.88      0.88      0.88        24



### Saving & loading back the trained model

In [22]:
clf_pipelineModel.stages

[DocumentAssembler_9bfba891e1c7,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 FinanceClassifierDLModel_596e43f4c90c]

In [23]:
clf_pipelineModel.stages[-1].write().overwrite().save('Clf_Use')

In [24]:
# Load back  saved Classifier Model
ClfModel = finance.ClassifierDLModel.load('Clf_Use')

In [25]:
ld_pipeline = nlp.Pipeline(stages=[document_assembler, embeddings, ClfModel])
ld_pipeline_model = ld_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [26]:
# Apply Model Transform to testData
ld_preds = ld_pipeline_model.transform(test)

In [27]:
ld_preds_df = ld_preds.select("text", "label", "class.result").toPandas()

In [28]:
ld_preds_df.head()

,text,label,result
0,\nPursuant to the requirements of the Securiti...,work_experience,[work_experience]
1,\nWe plan to continue to grow our business by ...,other,[other]
2,Goren is retained\nby the Company through an A...,work_experience,[work_experience]
3,"In December 2005, Ducati returned to Italian o...",work_experience,[other]
4,Weaver was Senior Vice President and Deputy Ge...,work_experience,[work_experience]


### Train with Bert Embeddings

We do not have Financial Sentence Embeddings yet, But we can use the Financial Word Embeddings and then average them. Since this model takes a long time to train, we will train for only one epoch.

In [29]:
embeddings = (
    nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base", "en")
    .setInputCols(["document", "token"])
    .setOutputCol("embeddings")
)

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[OK!]


In [30]:
document_assembler = (
    nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")
)

tokenizer = nlp.Tokenizer().setInputCols(["document"]).setOutputCol("token")

embeddingsSentence = (
    nlp.SentenceEmbeddings()
    .setInputCols(["document", "embeddings"])
    .setOutputCol("sentence_embeddings")
    .setPoolingStrategy("AVERAGE")
)

classifierdl = (
    finance.ClassifierDLApproach()
    .setInputCols(["sentence_embeddings"])
    .setOutputCol("class")
    .setLabelColumn("label")
    .setMaxEpochs(1)
    .setLr(0.001)
    .setEnableOutputLogs(True)
    .setOutputLogsPath("binary_bert")
    .setBatchSize(4)
    .setDropout(0.15)
)

clf_pipeline = nlp.Pipeline(
    stages=[document_assembler, tokenizer, embeddings, embeddingsSentence, classifierdl]
)

In [31]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 1.56 s, sys: 236 ms, total: 1.8 s
Wall time: 4min 33s


In [32]:
preds = clf_pipelineModel.transform(test)

In [33]:
preds_df = preds.select("label", "text", "class.result").toPandas()

In [34]:
preds_df.head()

,label,text,result
0,work_experience,\nPursuant to the requirements of the Securiti...,[work_experience]
1,other,\nWe plan to continue to grow our business by ...,[other]
2,work_experience,Goren is retained\nby the Company through an A...,[work_experience]
3,work_experience,"In December 2005, Ducati returned to Italian o...",[other]
4,work_experience,Weaver was Senior Vice President and Deputy Ge...,[work_experience]


In [35]:
log_files = os.listdir("binary_bert")

with open("binary_bert/"+log_files[0], "r") as log_file :
    print(log_file.read())

Training started - epochs: 1 - learning_rate: 0.001 - batch_size: 4 - training_examples: 289 - classes: 2
Epoch 0/1 - 0.68s - loss: 43.47585 - acc: 0.7465278 - batches: 73



In [36]:
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))


                 precision    recall  f1-score   support

          other       0.87      0.81      0.84        16
work_experience       0.67      0.75      0.71         8

       accuracy                           0.79        24
      macro avg       0.77      0.78      0.77        24
   weighted avg       0.80      0.79      0.79        24



### Save model and Zip it for Modelshub Upload/Downloads

In [ ]:
# Save a Spark NLP model
clf_pipelineModel.stages[-1].write().overwrite().save('ClfBert')

# cd into saved dir and zip
! cd /content/ClfBert ; zip -r /content/ClfBert.zip *